In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2


In [8]:
from vpm_py.visualization import Visualizer, SliceFilter_3D, ValueFilter
from vpm_py.visualization import SliceStrategy
import os
from vpm_py.visualization import ResultPlot
import numpy as np


In [ ]:
z_slice = SliceFilter_3D(strategy=SliceStrategy.MAX_INTEGRAL, plane="Z")
y_slice = SliceFilter_3D(strategy=SliceStrategy.MAX_INTEGRAL, plane="Y")
value_filter = ValueFilter(tolerance=1.5e-3, value=0.0, type="greater")

plot_particles = ResultPlot(
    "particle",
    quantity="charge",
    component="magnitude",
    filters=[],
)

plot_mesh_quantity_z = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[z_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)
plot_mesh_quantity_y = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[y_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)

plot_pressure_z = ResultPlot(
    "mesh",
    quantity="pressure",
    component="P",
    filters=[z_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)
plot_pressure_y = ResultPlot(
    "mesh",
    quantity="pressure",
    component="P",
    filters=[y_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)


plotter = Visualizer(
    figure_size=(12, 10),
    plot_options=[
        plot_particles,
        plot_mesh_quantity_z,
        plot_mesh_quantity_y,
        plot_pressure_z,
        plot_pressure_y,
    ],
)


In [13]:
# folder = 'results_test/'
# folder = '/mnt/c/Users/tryfonas/Data/build_exe/results'
folder = "/mnt/c/Users/tryfonas/Data/hill_vortex/results"
folder = "../build/release/gnu/bin/results_deform_dt=0.01/results"
# folder = "../build/debug/gnu/bin/results_deform_CFL_variable/results"
# folder = "/mnt/c/Users/tryfo/Videos/CFL_deform/results_deform_dt=0.01/results"
folder = '/mnt/c/Users/tryfonas/Data/hill_vortex_Re=100.0_nocorrect_remesh_40/results'



In [14]:
# plotter.add_folder(
#     folder=folder,
#     particle_filename_pattern= r".*particle.*\.h5",
#     mesh_filename_pattern= r".*pm.*\.h5",
# )


In [ ]:
import re

files = os.listdir(folder)
particle_pattern = r".*particles.*\.h5"
mesh_pattern = r".*mesh.*\.h5"
particle_regex = re.compile(particle_pattern)
mesh_regex = re.compile(mesh_pattern)

files_part = [f for f in files if particle_regex.match(f)]
files_mesh = [f for f in files if mesh_regex.match(f)]

total_frames = len(files_part)
for idx in np.linspace(0, len(files_part) - 1, 20, dtype=int):
    print(f"Loading frame {idx}")
    file_mesh = os.path.join(folder, files_mesh[idx])
    file_part = os.path.join(folder, files_part[idx])
    plotter.load_results_from_disk(idx, total_frames, file_part, file_mesh)

# plotter.animate_result_folder(
#     folder,
#     total_frames=len(files_part),
#     start_frame=0,
#     dt=0.1,
#     save_filename="animation_only_charged_particles",
#     format="mp4",
#     particle_filename_pattern=particle_pattern,
#     mesh_filename_pattern=mesh_pattern,
# )
